In [36]:
import pandas as pd
import numpy as np
import wandb

In [37]:
idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')

In [44]:
from sklearn.preprocessing import StandardScaler

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)
# , random_state=42

from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(n_splits=4,max_train_size=10000)

all_splits = list(ts_cv.split(X_train, y_train))
train_0, test_0 = all_splits[0]

In [45]:
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import wandb
import wandb

sweep_configuration_rfc = {
    "project": "RandomForestClass",
    "name": "my-awesome-sweep",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "n_estimators": {
            "values": [8, 16, 32, 64, 128, 256]
        },
        "criterion": {
            "values": ['entropy','gini']
        },
        "max_depth": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, None]
        },
        "bootstrap": {
            "values": [True, False]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30]
        },
        "min_samples_split": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30]
        }
    }
}

def my_train_func():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestClassifier(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split)

    model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)

    acc = zero_one_loss(y_test.ravel(), y_pred.ravel())
    loss = accuracy_score(y_test.ravel(), y_pred.ravel())

    wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    wandb.log({"feature_imp": wandb.sklearn.plot_feature_importances(model, Predictors)})
    wandb.log({"loss": loss})


# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration_rfc, project="RandomForestClass")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)


Create sweep with ID: rewlbkqu
Sweep URL: https://wandb.ai/jonathanweske/RandomForestClass/sweeps/rewlbkqu


wandb: Agent Starting Run: 7mzyxy3h with config:
wandb: 	bootstrap: False
wandb: 	criterion: entropy
wandb: 	max_depth: 40
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 6
wandb: 	min_samples_split: 18
wandb: 	n_estimators: 16


accuracy,▁
loss,▁
accuracy,0.41903
loss,0.58097


wandb: Agent Starting Run: l3pqcjzj with config:
wandb: 	bootstrap: True
wandb: 	criterion: entropy
wandb: 	max_depth: 20
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 5
wandb: 	min_samples_split: 11
wandb: 	n_estimators: 16


accuracy,▁
loss,▁
accuracy,0.43191
loss,0.56809


...

accuracy,▁
loss,▁
accuracy,0.44377
loss,0.55623


wandb: Agent Starting Run: 3w67ejan with config:
wandb: 	bootstrap: False
wandb: 	criterion: gini
wandb: 	max_depth: None
wandb: 	max_features: auto
wandb: 	min_samples_leaf: 3
wandb: 	min_samples_split: 7
wandb: 	n_estimators: 16


accuracy,▁
loss,▁
accuracy,0.41982
loss,0.58018


wandb: Agent Starting Run: 31mes0wx with config:
wandb: 	bootstrap: True
wandb: 	criterion: entropy
wandb: 	max_depth: 20
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 25
wandb: 	n_estimators: 128


wandb: Ctrl + C detected. Stopping sweep.


#### CV cross_validate()
Mean Absolute Error:     0.672 +/- 0.019
Root Mean Squared Error: 1.057 +/- 0.016

#### cross_val_score
[0.49957698 0.53019349 0.54551058 0.49902538]
Accuracy of Model with Cross Validation is: 51.85766080195356